In [29]:
import requests
import re
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import time
import pandas as pd
from langchain_chroma import Chroma
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector
#from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import csv

In [30]:
# ONOS Controller Details
ONOS_BASE_URL = "http://10.23.7.63:8181/onos/v1"
USERNAME = "onos"
PASSWORD = "rocks"  # Replace with your ONOS credentials

# Headers for REST API
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Define sudo password
sudo_password = "test@irciss008"

In [31]:
def get_flows_for_device_ONOS(device_id):
    """
    Retrieve all flows for a specific ONOS device (switch).
    
    Parameters:
        device_id (str): The ONOS switch ID (e.g., "of:0000000000000002").
    
    Returns:
        list: A list of flow JSON objects for the given switch.
    """
    url = f"{ONOS_BASE_URL}/flows/{device_id}"
    headers = {"Accept": "application/json"}

    try:
        response = requests.get(url, auth=(USERNAME, PASSWORD), headers=headers)
        response.raise_for_status()
        
        data = response.json()
        return data.get("flows", [])

    except requests.exceptions.RequestException as e:
        print(f"Error fetching flows for device {device_id}:", e)
        return []

def extract_switch_id_ONOS(intent: str):
    """
    Extract the switch ID from a natural language intent for ONOS JSON format.
    
    Parameters:
        intent (str): The natural language intent.
    
    Returns:
        str: Extracted switch ID (e.g., 'of:0000000000000001') or None if not found.
    """
    # Mapping of ordinal words to numeric values
    ordinals = {
        "first": 1,
        "second": 2,
        "third": 3,
        "fourth": 4,
        "fifth": 5,
        "sixth": 6,
        "seventh": 7,
        "eighth": 8,
        "ninth": 9,
        "tenth": 10
    }

    # Match patterns like 'switch 1', 'router 2', 'node 3'
    match = re.search(r'\b(?:switch|router|node|device)(?:\s*number)?\s*(\d+)', intent, re.IGNORECASE)
    if match:
        switch_number = int(match.group(1))
        return f"of:{switch_number:016x}"  # Convert to ONOS 16-digit hex format

    # Match ordinal words (e.g., 'fourth switch', 'second router')
    match = re.search(r'\b(?:switch|router|node|device)\s*(\w+)', intent, re.IGNORECASE)
    if match:
        ordinal_word = match.group(1).lower()
        if ordinal_word in ordinals:
            switch_number = ordinals[ordinal_word]
            return f"of:{switch_number:016x}"  # Convert to ONOS 16-digit hex format

    # Match standalone ordinal words (e.g., 'fourth' without 'switch')
    for word, number in ordinals.items():
        if word in intent.lower():
            return f"of:{number:016x}"  # Convert to ONOS 16-digit hex format

    return None

In [32]:
TRANSLATION_PROMPT_ONOS = """Your task is to transform natural language network intents into JSON-formatted network policies compatible with the ONOS SDN controller.

You only reply in JSON, no natural language. The network intents can represent different traffic control behaviors, such as:

1. **Traffic Forwarding, Queue Assignment, and VLAN Rules:** Define rules for forwarding traffic based on IPv4/IPv6 destination, TCP/UDP ports, and optionally assign traffic to specific queues or vlans.
2. **Blocking or Dropping Rule:** Define rules to drop traffic based on specific match criteria (e.g., source IP, destination IP). In ONOS, this is done by omitting the `"treatment"` field.

### **JSON STRUCTUREs FOR ONOS**

1. **Traffic Forwarding, Queue Assignment, and VLAN Rules:**  

```json
{
    "flows": [
        {
            "priority": <integer>,
            "timeout": <integer>, // Default: 0
            "isPermanent": "true",
            "deviceId": "<switch_id>",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": <integer>
                    },
                    {
                        "type": "L2MODIFICATION",
                        "subtype": "VLAN_ID",
                        "vlanId": <integer> // Example: 100 for VLAN tagging
                    },
                    {
                        "type": "OUTPUT",
                        "port": "<integer>"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "<string>" // Example: "0x800" for IPv4
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": <integer> // Example: 6 for TCP, 17 for UDP
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": <integer>
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": <integer>
                    },
                    {
                        "type": "IN_PORT",
                        "port": "<integer>"
                    }
                ]
            }
        }
    ]
}


2. **Blocking or Dropping Rule:**

{
    "flows": [
        {
            "priority": <integer>,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "<switch_id>",
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "<string>" // Example: "0x800" for IPv4
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "<ip_address/mask>"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "<ip_address/mask>"
                    }
                ]
            }
        }
    ]
}

Field Descriptions
priority (Mandatory): Priority level (higher numbers indicate higher priority). For blocking or firewall rules, assign a priority greater than 300.
timeout (Mandatory): Timeout in seconds after which the flow is removed (Default: 0).
isPermanent (Mandatory): "true" (always in quotes, per user preference).
deviceId (Mandatory): Switch ID where the rule is installed.
ethType (Mandatory): Ethernet protocol type. Use "0x800" for IPv4, "0x86DD" for IPv6, "0x806" for ARP.
IPV4_DST (Optional): IPv4 address in CIDR notation (e.g., "10.0.0.1/32"). Include only if explicitly mentioned.
IPV4_SRC (Optional): Source IP address (include only if explicitly mentioned).
IP_PROTO (Optional): Transport layer protocol (6 for TCP, 17 for UDP, 1 for ICMP).
TCP_DST (Optional): TCP destination port (e.g., 80 for HTTP).
UDP_DST (Optional): UDP destination port (e.g., 161 for SNMP).
IN_PORT (Optional): Incoming interface port number (use in port-based forwarding).
QUEUE (Optional): Use "QUEUE" with "queueId" to specify a QoS queue (queue ID is an integer, 0 is default).
OUTPUT (Optional): "OUTPUT" with "port" specifies the output port.
VLAN_ID (Optional): Use "L2MODIFICATION" with "subtype": "VLAN_ID" and "vlanId" to set a VLAN tag.

Rules for Translation
Each "priority" must be unique.
Set priority high (e.g., 1000) for queue-related rules.
Do not include VLAN-related fields unless explicitly mentioned in the intent.
Do not include optional fields unless explicitly mentioned in the intent.
Ensure valid ONOS-compliant JSON syntax.
Verify JSON structure before responding.
Always respond in valid JSON format only, without comments, explanations, or additional text.
If the intent cannot be mapped, return an empty JSON object {}."""

In [33]:
SLICING_PROMPT = """You are tasked with analyzing a natural language intent to determine if it contains a command to create or use a queue/slice in a openflow switch. You should respond in JSON format. 

### Rules for Interpretation:
1. **Queue/Slice Detection:**  
   - The intent is considered related to queue/slice if it contains commands such as:
     - "create queue", "create slices", "slice the network", "implement slicing", "slice the flow", "make flowspace slicing", "do slicing", "slice", "implement queue", "do queuing", "assign queue", "assign slice", or any similar phrasing.
   - If the intent does not mention creating or using a queue/slice, set the field `"use_queue"` to `0`. 

2. **Switch and Port Identification:**  
   - If the intent specifies a **switch ID** (e.g., "switch 4" or "openflow:4" or "node 4" or "openflow 4"), populate the `"switch_id"` field with its value.  
   - If the intent specifies a **Queue ID or slice ID** (e.g., "queue 4" or "4th queue" or "fourth queue" or "slice 1" or "first slice"), populate the `"queue_id"` field with its value.  
   - If the intent specifies a **port ID** (e.g., "port 2" or "interface 2" or "ethernet 2" or "output node connector 2" or "second port" or "second interface"), populate the `"port_id"` field with its value. If there are multiple instances of "port_id" present, take the one which indicates output port or outgoing interface.
   - If the intent does not specify a switch ID or queue ID or port ID, set the respective field to an empty string (`""`).

3. **Output Format:**  
   - Respond strictly in valid JSON format adhering to the following schema:

```json
{
  "use_queue": <integer>,
  "switch_id": "<string>",
  "queue_id": "<string>",
  "port_id": "<string>"
}

Field Description:
-use_queue: 1 if the intent commands to create or use a queue/slice, 0 otherwise.
-switch_id: Switch ID if specified in the intent, otherwise "".
-queue_id: Queue ID if specified in the intent, otherwise "".
-port_id: Port ID if specified in the intent, otherwise "". If there are mutiple instances of "port_id" present, take the one which indicates the output port or outgoing interface.

4. No Additional Text:
Do not include any comments, explanations, or outputs outside the JSON format.

Example Inputs and Outputs:
    1. Input Intent:
    "Create a queue in switch 4 on port 3 for slicing the flow."
    Output:
    {
    "use_queue": 1,
    "switch_id": "switch 4",
    "queue_id": "",
    "port_id": "port 3"
    }

    2. Input Intent:
    "Send all video traffic through queue 0 of openflow:2."
    Output:
    {
    "use_queue": 1,
    "switch_id": "openflow 2",
    "queue_id": 0,
    "port_id": ""
    }

    3. Input Intent:
    "Configure switch 5 for traffic management."
    Output:
    {
    "use_queue": 0,
    "switch_id": "switch 5",
    "queue_id": "",
    "port_id": ""
    }

    4. Input Intent:
    "Monitor traffic flow on port 1."
    Output:
    {
    "use_queue": 0,
    "switch_id": "",
    "queue_id": "",
    "port_id": "port 1"
    }

   5. Input Intent:
    "In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 0 for prioritized handling."
    Output:
    {
    "use_queue": 1,
    "switch_id": switch 3,
    "queue_id": 0,
    "port_id": "interface 2"
    }

Respond with the appropriate JSON strictly following these rules and format."""

In [34]:
CONFLICT_PROMPT_ONOS = """You are tasked with determining if two ONOS flow configuration JSONs directly conflict with each other. **You MUST base your decision SOLELY on the JSON content provided.** Do **NOT** infer intent, use semantic reasoning, or make assumptions beyond the given JSON structure.

---

### **Rules for Conflict Detection (STRICT and LITERAL COMPARISON):**

A **direct conflict** exists **ONLY IF** all the following conditions are met:

#### **1. Matching Traffic Characteristics (Exact or Overlapping Match Criteria)**
Both flows must match **identical or overlapping traffic characteristics** for all the specified fields, meaning they could apply to the **exact same packets**. The following fields are checked:

**EtherType (`ETH_TYPE`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `"0x800"` for IPv4, `"0x86DD"` for IPv6).
   - If missing in either flow, there is **no match**.

**Source and Destination IP (`IPV4_SRC` and `IPV4_DST`)**  
   - If both flows **specify** a source or destination, they must overlap (e.g., `"10.0.0.1/32"` and `"10.0.0.0/24"` overlap).
   - If **either** flow **omits** source or destination, it applies to **all** sources or destinations respectively.
   - If one flow has `IPV4_SRC` and the other **does not**, it is **not a conflict**.

**Protocol (`IP_PROTO`)**  
   - Must be present in **both** flows and have **identical** values (e.g., `6` for TCP, `17` for UDP).
   - If missing in either flow, there is **no match**.

**Transport Layer Ports (`TCP_SRC`, `TCP_DST`, `UDP_SRC`, `UDP_DST`)**  
   - Must be **identical** in both flows if present.
   - If one flow includes a port filter and the other does not, there is **no match**.

**Incoming Port (`IN_PORT`)**  
   - If both flows specify `IN_PORT`, they must be identical.
   - If missing in either flow, there is **no match**.

**Wildcard Matching (Implicit Behavior for Missing Fields)**  
   - If a flow **specifies a field** (e.g., `IPV4_SRC`), it applies **only** to that source.
   - If a flow **does not specify** `IPV4_SRC`, it applies to **all sources**.
   - A specific rule does **not** conflict with a general rule unless the **actions contradict**.

**EXCEPTION for Additional Match Fields:**  
   - If one flow has **extra** match fields (e.g., protocol, port, VLAN) and the other does **not**, they do **not** conflict even if the output actions differ.

---

#### **2. Contradictory Actions**
If the flows **match the same traffic**, they **only** conflict if their **actions are contradictory**:

**Different Output Ports (`OUTPUT` instruction)**  
   - If one flow **forwards** traffic to port `X` and another to port `Y`, it is a **conflict**.

**One Flow Drops, One Flow Forwards**  
   - If one flow **omits `"treatment"`** (implying a drop) and another **forwards traffic**, it is a **conflict**.

**Different Queue Assignments (`QUEUE` instruction)**  
   - If the flows **assign different `queueId` values**, it is a **conflict**.

**EXCEPTION for Additional Match Fields:**  
   - If one flow has **extra** match fields (e.g., `TCP_DST`, `UDP_DST`) and the other does not, **there is no conflict**.

---

#### **3. Priority is Irrelevant for Conflict Detection**
- `priority` **does not** determine a conflict.
- Even if one rule **overrides** another, they **do not conflict** unless their actions contradict.

---

#### **4. General vs. Specific Rules**
A **general rule** does **not** conflict with a **specific rule** unless:
- The specific rule **fully overlaps** with the general rule’s match criteria.
- The actions contradict.

**NO CONFLICT Example:**  
   - Flow 1: `{ "match": { "IPV4_DST": "10.0.0.2/32" }, "treatment": { "instructions": [ { "type": "OUTPUT", "port": "3" } ] } }`
   - Flow 2: `{ "match": { "IPV4_SRC": "10.0.0.1/32", "IPV4_DST": "10.0.0.2/32" }, "treatment": { "instructions": [ { "type": "OUTPUT", "port": "3" } ] } }`
   - **Explanation:** Flow 1 applies to **all sources**, while Flow 2 applies **only** to traffic from `10.0.0.1`. Since both **output to port 3**, there is **no conflict**.

**CONFLICT Example (Contradictory Actions for Same Traffic)**  
   - Flow 1: `{ "match": { "IPV4_SRC": "192.168.1.1/32", "IPV4_DST": "10.0.0.2/32" }, "treatment": { "instructions": [ { "type": "OUTPUT", "port": "3" } ] } }`
   - Flow 2: `{ "match": { "IPV4_SRC": "192.168.1.1/32", "IPV4_DST": "10.0.0.2/32" } }`
   - **Explanation:** Flow 1 forwards the traffic, while Flow 2 **drops it** (no `"treatment"` field). **This is a conflict**.

---

### **Input Format**
You will be provided with **two ONOS JSON flow rules** in the following format:

**Flow 1:**
```json
<JSON for Flow 1>

**Flow 2:**
<JSON for Flow 2>

### **Expected Output Format**
Respond strictly in valid JSON format, using the schema below:

{
    "conflict_status": <integer>,
    "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:
   - conflict_status should be 1 if a direct conflict exists, 0 otherwise.
   - conflict_explanation → A brief explanation if a conflict exists, otherwise an empty string "".

NO EXTRA TEXT, COMMENTS, OR EXPLANATIONS OUTSIDE JSON.
DO NOT return 1 unless you are certain of a direct conflict.
Follow strict field-by-field comparison rules—NO inference beyond given data."""

In [ ]:
my_models_translate_real = [
"codestral"
]

my_models_conflict_real = [
"marco-o1",
"mistral",
"mistral-nemo",
"deepseek-coder",
"starcoder", 
"codegemma",
"starcoder2",
"openchat",
"phi3",
"dolphin-mistral",
"wizardlm2",
"phi",
"yi",
"zephyr",
"command-r",
"llava-llama3",
"codestral",
"codellama:34b",
"codellama",
"llama2",
"llama3",
"llama3.1",
"llama3.2",
"qwen",
"qwen2",
"qwen2.5",
"gemma2:27b",
"huihui_ai/qwq-abliterated",
"huihui_ai/qwq-fusion",
"qwq",
"llama3.3",
"llama2:70b",
"codellama:70b"
]

#"phi", "orca-mini", "qwen", "tinyllama", "deepseek-coder-V2", does not work

context_examples = [3, 6]

default_model = "llama2"

ollama_embedding_url = "http://localhost:11434"
ollama_server_url = "http://localhost:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [36]:
# Load custom dataset from CSV
custom_dataset = pd.read_csv('ONOS_intent_translation_dataset_for_LLM_Evaluation.csv')

# Ensure proper column names and format
if not {'instruction', 'output'}.issubset(custom_dataset.columns):
    raise ValueError("The dataset must have 'instruction' and 'output' columns.")

# Split into train and test (50/50 split for example)
#trainset, testset = train_test_split(custom_dataset, test_size=0.5, random_state=42, shuffle=True)
trainset = custom_dataset

In [37]:
# Define the file path
output_file = f"JSON_conflict_details_ONOS_70b_{int(time.time())}.txt"
# Open the file in write mode
logfile = open(output_file, "w")  # Use "w" for write mode

# Ground truth: Stores the actual conflicting rule indices for each intent and corresponding device
ground_truth_conflict_rules_dict = {
    "intent1": {2},    
    "intent2": {7}, 
    "intent3": {3},  
    "intent4": set(), 
    "intent5": {2, 4},
    "intent6": {2, 6},
    "intent7": {2},
    "intent8": {1},
    "intent9": set(),
    "intent10": {5}
}

# Initialize TP, FP, TN, FN for each model
model_metrics = {model: {"TP": 0, "FP": 0, "TN": 0, "FN": 0, "total_conflicts": 0, "avg_time": 0, "num_intents": 0} for model in my_models_conflict_real}

In [38]:
def run_LLM_conflict(existing_intent_flow_json, new_intent_flow_json, device_id, intent_name, nat_lang_intent):
    system_prompt = CONFLICT_PROMPT_ONOS
    result_dict_conflict_rules = {model: False for model in my_models_conflict_real}  # Stores if conflict is found per rule
    result_dict_time_avg = {model: 0 for model in my_models_conflict_real}

    for model in my_models_conflict_real:
        count = 0
        while True:
            count+=1
            try:
                time.sleep(0.1)
                current_time = time.time()

                response = client.generate(model=model,
                    options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                    stream=False,
                    system=system_prompt,
                    prompt=f"Flow 1:\n{json.dumps(existing_intent_flow_json, indent=2)}\n\nFlow 2:\n{json.dumps(new_intent_flow_json, indent=2)}",
                    format='json'
                )

                output = response['response'].strip()
                proc_time_s = (time.time() - current_time)

                response_json = json.loads(output)

                if 'conflict_status' not in response_json:
                    print("\nWarning: 'conflict_status' key is missing in the response.\n")

                conflict_status = response_json.get('conflict_status', 0)

                # Ensure conflict_status is an integer
                if isinstance(conflict_status, str):
                    conflict_status = int(conflict_status)

                if conflict_status == 1:  # If the model detected conflict
                    result_dict_conflict_rules[model] = True  # Mark this rule as conflicting

                result_dict_time_avg[model] += proc_time_s

                logfile.write(f"\n\n===========================================")
                logfile.write(f"\n{intent_name}\n{nat_lang_intent}")
                logfile.write(f"\nDevice id :{device_id}, Conflict status :{ground_truth_conflict_rules_dict[intent_name]}\n")
                logfile.write(f"Model name: {model}\n")
                logfile.write(f"Existing intent flow json:\n {existing_intent_flow_json}\n")
                logfile.write(f"New intent flow json:\n {new_intent_flow_json}\n")
                logfile.write(f"Conflict Info json:\n {response_json}\n")
                logfile.write(f"Processing time:\n {round(proc_time_s,2)}\n")
                break

            except Exception as e:
                print("Exception found: ", e)
                sys.stdout.flush()
                if(count<100):
                    continue
                else:
                    print("\n",model, " failed to produce valid JSON for conflict info after ", count ," tries. Going to next model\n")
                    break               
    
    return result_dict_conflict_rules, result_dict_time_avg

In [39]:
# Dictionary to store manually provided flows for each device ID
manual_flows_by_device = {
    "of:0000000000000001": {
        "flows": [
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "3"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.1/32"}
                ]
            }
            },
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "4"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.2/32"}
                ]
            }
            },
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "1"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.3/32"}
                ]
            }
            },
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "1"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.4/32"}
                ]
            }
            },

            {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 0
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 6
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.3/32"
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": 80
                    }
                ]
            }
        },
        {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 0
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 6
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.4/32"
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": 80
                    }
                ]
            }
        },
        {
            "priority": 499,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 1
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 17
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.3/32"
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": 80
                    }
                ]
            }
        },
        {
            "priority": 499,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000001",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 1
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 17
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.4/32"
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": 80
                    }
                ]
            }
        }
            
        ]
    },
    "of:0000000000000002": {
        "flows": [
            {
            "priority": 2,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000002",
            "treatment": {
                "instructions": [
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "IN_PORT",
                        "port": 1
                    }
                ]
            }
        },
        {
            "priority": 2,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000002",
            "treatment": {
                "instructions": [
                    {
                        "type": "OUTPUT",
                        "port": "1"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "IN_PORT",
                        "port": 2
                    }
                ]
            }
        }

        ]
    },
    "of:0000000000000003": {
        "flows": [
            {
            "priority": 2,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000003",
            "treatment": {
                "instructions": [
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "IN_PORT",
                        "port": 1
                    }
                ]
            }
        },
        {
            "priority": 2,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000003",
            "treatment": {
                "instructions": [
                    {
                        "type": "OUTPUT",
                        "port": "1"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "IN_PORT",
                        "port": 2
                    }
                ]
            }
        },

        {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000003",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 0
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 6
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": 80
                    },
                    {
                        "type": "IN_PORT",
                        "port": 1
                    }
                ]
            }
        },
        {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000003",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 0
                    },
                    {
                        "type": "OUTPUT",
                        "port": "1"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 6
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": 80
                    },
                    {
                        "type": "IN_PORT",
                        "port": 2
                    }
                ]
            }
        },
        {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000003",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 1
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 17
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": 80
                    },
                    {
                        "type": "IN_PORT",
                        "port": 1
                    }
                ]
            }
        },
        {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000003",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 1
                    },
                    {
                        "type": "OUTPUT",
                        "port": "1"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 17
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": 80
                    },
                    {
                        "type": "IN_PORT",
                        "port": 2
                    }
                ]
            }
        }   
            
        ]
    },
    "of:0000000000000004": {
        "flows": [
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "3"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.3/32"}
                ]
            }
            },
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "4"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.4/32"}
                ]
            }
            },
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "1"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.1/32"}
                ]
            }
            },
            {
            "priority": 102,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {"type": "OUTPUT", "port": "1"}
                ]
            },
            "selector": {
                "criteria": [
                    {"type": "ETH_TYPE", "ethType": "0x0800"},
                    {"type": "IPV4_DST", "ip": "10.0.0.2/32"}
                ]
            }
            },

            {
            "priority": 1000,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IPV4_SRC",
                        "ip": "10.0.0.1/32"
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.4/32"
                    }
                ]
            }
            },

            
            {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 0
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 6
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.1/32"
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": 80
                    }
                ]
            }
        },
        {
            "priority": 500,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 0
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 6
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.2/32"
                    },
                    {
                        "type": "TCP_DST",
                        "tcpPort": 80
                    }
                ]
            }
        },
        {
            "priority": 499,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 1
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 17
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.1/32"
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": 80
                    }
                ]
            }
        },
        {
            "priority": 499,
            "timeout": 0,
            "isPermanent": "true",
            "deviceId": "of:0000000000000004",
            "treatment": {
                "instructions": [
                    {
                        "type": "QUEUE",
                        "queueId": 1
                    },
                    {
                        "type": "OUTPUT",
                        "port": "2"
                    }
                ]
            },
            "selector": {
                "criteria": [
                    {
                        "type": "ETH_TYPE",
                        "ethType": "0x800"
                    },
                    {
                        "type": "IP_PROTO",
                        "protocol": 17
                    },
                    {
                        "type": "IPV4_DST",
                        "ip": "10.0.0.2/32"
                    },
                    {
                        "type": "UDP_DST",
                        "udpPort": 80
                    }
                ]
            }
        }
        ]
    }
}

In [40]:
def conflict(device_id, new_intent_flow_json, intent_name, nat_lang_intent):
    
    #existing_flows = get_flows_for_device_ONOS(device_id)
    existing_flows = manual_flows_by_device.get(device_id, {}).get("flows", [])

    total_rules = len(existing_flows)  # Number of flow rules in the switch

    model_results = {model: set() for model in my_models_conflict_real}  # Store detected conflicting rule indices
    model_times = {model: 0 for model in my_models_conflict_real}  # Store time taken per model

    # Get ground truth conflicting rule positions
    ground_truth_conflict_rules = ground_truth_conflict_rules_dict[intent_name]

    for rule_index, existing_flow in enumerate(existing_flows, start=1):
        model_conflict_results, model_time_avg = run_LLM_conflict(existing_flow, new_intent_flow_json, device_id, intent_name, nat_lang_intent)

        # Aggregate detected conflicts per model
        for model in my_models_conflict_real:
            if model_conflict_results[model]:  # If model found a conflict
                model_results[model].add(rule_index)  # Store conflicting rule position
            model_times[model] += model_time_avg[model]

    # Compute TP, FP, TN, FN for each model
    for model in my_models_conflict_real:
        predicted_conflict_rules = model_results[model]  # Model's detected conflicting rules

        TP = len(predicted_conflict_rules & ground_truth_conflict_rules)  # Correct rule predictions
        FP = len(predicted_conflict_rules - ground_truth_conflict_rules)  # Wrong rule predictions
        FN = len(ground_truth_conflict_rules - predicted_conflict_rules)  # Missed rules
        TN = total_rules - (TP + FP + FN)  # Remaining correctly identified non-conflicting rules

        model_metrics[model]["TP"] += TP
        model_metrics[model]["FP"] += FP
        model_metrics[model]["TN"] += TN
        model_metrics[model]["FN"] += FN

        # Track processing time
        model_metrics[model]["total_conflicts"] += len(predicted_conflict_rules)
        model_metrics[model]["avg_time"] += model_times[model]
        model_metrics[model]["num_intents"] += 1

In [41]:
def run_LLM_IBN(intent, device_id):

    for num_examples in context_examples:
        for model in my_models_translate_real:
            
            example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
                [{"instruction": trainset.iloc[0]["instruction"], "output": trainset.iloc[0]["output"]}],
                ollama_emb,
                Chroma,
                input_keys=["instruction"],
                k=num_examples,
                vectorstore_kwargs={"fetch_k": min(num_examples, len(trainset))}
                )
            # Clear and add all remaining examples from the trainset
            example_selector.vectorstore.reset_collection()
            
            for _, row in trainset.iterrows():
                example_selector.add_example({
                    "instruction": row["instruction"],
                    "output": row["output"]
                })
            
            system_prompt = TRANSLATION_PROMPT_ONOS
            count = 0

            while True:
                count+=1
                try:
                    time.sleep(0.1)
                    if num_examples > 0:
                        examples = example_selector.select_examples({"instruction": intent})
                        example_str = "\n\n\n".join(map(lambda x: "Input: " + x["instruction"] + "\n\nOutput: " + x["output"], examples))
                        system_prompt += example_str + "\n\n\n"  
                    
                    response = client.generate(model=model,
                        options={'temperature': 0.6, 'num_ctx': 8192, 'top_p': 0.3, 'num_predict': 1024, 'num_gpu': 99},
                        #options={'device': 'cpu'},
                        stream=False,
                        system=system_prompt,
                        prompt=intent,
                        format='json'
                    )
                    actual_output = response['response']
                    print("\nTranslated by: ", model)
                    break
                
                except Exception as e:
                    print("Exception on Input: ", e)
                    print("\nCheck example_str same or not: \n",example_str)
                    sys.stdout.flush()
                    if(count<15):
                        continue
                    else:
                        print("\n",model, " failed to produce valid JSON for translation info after 15 tries. Going to next model\n")
                        break 
            try:
                
                flow_json = json.loads(actual_output)
                
                for flow in flow_json.get("flows", []):  # Iterate over all flows
                    flow["deviceId"] = device_id  # Replace the device ID

                return flow_json
            
            except Exception as e:
                print("Exception found: ", e)

In [42]:
intent1 = "In switch 4, install a firewall to block traffic from 10.0.0.2 to 10.0.0.4."
device_id1 = extract_switch_id_ONOS(intent1)
intent1_JSON = run_LLM_IBN(intent1, device_id1)

intent2 = "Using openflow switch number 1, forward UDP traffic on port 80 destined for 10.0.0.3 via interface 2, assigning it to queue 0 for prioritized handling."
device_id2 = extract_switch_id_ONOS(intent2)
intent2_JSON=run_LLM_IBN(intent2, device_id2)

intent3 = "In switch 3, if the incoming traffic in port 1 is TCP traffic destined for port 80, then pass it via interface 2, assigning it to queue 1 for prioritized handling."
device_id3 = extract_switch_id_ONOS(intent3)
intent3_JSON=run_LLM_IBN(intent3, device_id3)

intent4 = "If the incoming traffic in interface 3 of node 3 is UDP traffic destined for port 80, then pass it via port 2, assigning it to queue 1 for prioritized handling."
device_id4 = extract_switch_id_ONOS(intent4)
intent4_JSON=run_LLM_IBN(intent4, device_id4)

intent5 = "In switch 3, if the incoming traffic in port 2 is TCP traffic destined for port 80, then pass it via interface 3, assigning it to queue 0 for prioritized handling."
device_id5 = extract_switch_id_ONOS(intent5)
intent5_JSON=run_LLM_IBN(intent5, device_id5)

intent6 = "If the incoming traffic in interface 2 of switch 3 is UDP traffic destined for port 80, then pass it via port 1, assigning it to queue 0 for prioritized handling."
device_id6 = extract_switch_id_ONOS(intent6)
intent6_JSON=run_LLM_IBN(intent6, device_id6)

intent7 = "In node 1, traffic destined for 10.0.0.2 should use port 3"
device_id7 = extract_switch_id_ONOS(intent7)
intent7_JSON=run_LLM_IBN(intent7, device_id7)

intent8 = "In switch 2, traffic from port 1 should pass through port 3."
device_id8 = extract_switch_id_ONOS(intent8)
intent8_JSON=run_LLM_IBN(intent8, device_id8)

intent9 = "Traffic from port 2 of switch 2 destined for 10.0.0.1 should use interface 4 for passing."
device_id9 = extract_switch_id_ONOS(intent9)
intent9_JSON=run_LLM_IBN(intent9, device_id9)

intent10 = "In switch 4, traffic from 10.0.0.1 to 10.0.0.4 should use output interface 4."
device_id10 = extract_switch_id_ONOS(intent10)
intent10_JSON=run_LLM_IBN(intent10, device_id10)


conflict(device_id1, intent1_JSON, "intent1", intent1)
conflict(device_id2, intent2_JSON, "intent2", intent2)
conflict(device_id3, intent3_JSON, "intent3", intent3)
conflict(device_id4, intent4_JSON, "intent4", intent4)
conflict(device_id5, intent5_JSON, "intent5", intent5)
conflict(device_id6, intent6_JSON, "intent6", intent6)
conflict(device_id7, intent7_JSON, "intent7", intent7)
conflict(device_id8, intent8_JSON, "intent8", intent8)
conflict(device_id9, intent9_JSON, "intent9", intent9)
conflict(device_id10, intent10_JSON, "intent10", intent10)

# Compute average processing time per model
for model in my_models_conflict_real:
    if model_metrics[model]["num_intents"] > 0:
        model_metrics[model]["avg_time"] /= model_metrics[model]["num_intents"]

# Save results to CSV
csv_filename = f"JSON_conflict_accuracies_ONOS_70b_{int(time.time())}.csv"
with open(csv_filename, mode='w', newline='') as file:
    fieldnames = ["Model", "TP", "FP", "TN", "FN", "Total Conflicts Detected", "Avg Time"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for model, metrics in model_metrics.items():
        writer.writerow({
            "Model": model,
            "TP": metrics["TP"],
            "FP": metrics["FP"],
            "TN": metrics["TN"],
            "FN": metrics["FN"],
            "Total Conflicts Detected": metrics["total_conflicts"],
            "Avg Time": round(metrics["avg_time"], 2)
        })

print(f"Results saved to {csv_filename}")

logfile.close()


Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral

Translated by:  codestral
Exception found:  llama runner process has terminated: error loading model: unable to allocate backend buffer
llama_load_model_from_file: failed to load model
Exception found:  llama runner process has terminated: error loading model: unable to allocate backend buffer
llama_load_model_from_file: failed to load model
Exception found:  llama runner process has terminated: error loading model: unable to allocate backend buffer
llama_load_model_from_file: failed to load model
Exception found:  llama runner process has terminated: error loading model: unable to allocate backend buffer
llama_load_model_from_file: failed to load model
Exception found:  llama runner process has terminated: error loading model: unable to allocate

In [43]:
# task 1
# conflict found with same in out port but with different IP protocol and port and priority.
# Needs fixing.
